# Neighbourhood clustering

## Part 1: Setting up DataFrame

### Import necessary libraries

In [1]:
import pandas as pd

### First we scrape the data from the table


In [2]:
df_web = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')

In [3]:
dfToronto=df_web[0]

In [4]:
dfToronto.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


### We see some data entries which are invalid, let's clean up the DataFrame

In [5]:
dfToronto=dfToronto.drop(dfToronto[dfToronto['Borough']=='Not assigned'].index)

In [6]:
dfToronto[dfToronto['Postal Code'].duplicated(keep=False)]  ##No repeating postal codes

,Postal Code,Borough,Neighborhood


### Check size of cleaned DataFrame

In [7]:
dfToronto.shape

(103, 3)

## Part 2: Obtaining geolocation info


### Install Geocoder & Geopy

In [86]:
pip install geopy

Note: you may need to restart the kernel to use updated packages.


In [87]:
pip install geocoder

Note: you may need to restart the kernel to use updated packages.


In [8]:
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values


In [9]:
CLIENT_ID = '3B12YESKZWGLATGW0PZ10F5GLGQSCNLCV1ADBAH05ZSMMXEI' # your Foursquare ID
CLIENT_SECRET = 'DBV040MHE1HLQJSMD0H4JKROPLB1FFBXQXJ00FKMDOBVM40M' # your Foursquare Secret
VERSION = '20180606'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 3B12YESKZWGLATGW0PZ10F5GLGQSCNLCV1ADBAH05ZSMMXEI
CLIENT_SECRET:DBV040MHE1HLQJSMD0H4JKROPLB1FFBXQXJ00FKMDOBVM40M


In [10]:
address = 'Waterloo, ON'

geolocator = Nominatim(user_agent="meep_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

43.466874 -80.524635


In [11]:
dfToronto

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"
11,M3B,North York,Don Mills
12,M4B,East York,"Parkview Hill, Woodbine Gardens"
13,M5B,Downtown Toronto,"Garden District, Ryerson"


In [43]:
dfToronto=dfToronto.replace(to_replace ="Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",  
                            value ="969 Eastern Ave, South Central Letter Processing Plant Toronto")

dfToronto=dfToronto.replace(to_replace ="Stn A PO Boxes",value ="25 The Esplanade")

dfToronto=dfToronto.replace(to_replace ="Canada Post Gateway Processing Centre",  
                            value ="Canada Post Gateway")

dfToronto=dfToronto.replace(to_replace ="Studio District",  
                            value ="Distillery District")
dfToronto=dfToronto.replace(to_replace ="Del Ray, Mount Dennis, Keelsdale and Silverthorn",  
                            value ="Mount Dennis, Del Ray, Keelsdale and Silverthorn")

In [44]:
sep = ','
lat=[]
long=[]
for i in dfToronto[['Borough','Neighborhood']].values:
    location = geolocator.geocode(('Ontario, '+i[0]+', '+i[1].split(sep,-1)[0]))
    if(location):
        print(('Ontario, '+i[0]+', '+i[1].split(sep,-1)[0]))
        print(location.latitude,location.longitude,'\n')
        lat.append(location.latitude)
        long.append(location.longitude)
    else:
        print(i)
        try:
            print(('Ontario, '+i[0].split(' ',-1)[1]+', '+i[1].split(sep,-1)[0]))
            location = geolocator.geocode(('Ontario, '+i[0].split(' ',-1)[1]+', '+i[1].split(sep,-1)[0]))
        except:
            print(('Ontario, '+i[0]+', '+i[1].split('-',-1)[0]))
            location = geolocator.geocode(('Ontario, '+i[0]+', '+i[1].split('-',-1)[0]))
        print(location.latitude,location.longitude,'\n')
        lat.append(location.latitude)
        long.append(location.longitude)

Ontario, North York, Parkwoods
43.7587999 -79.3201966 

Ontario, North York, Victoria Village
43.732658 -79.3111892 

['Downtown Toronto' 'Regent Park, Harbourfront']
Ontario, Toronto, Regent Park
43.6607056 -79.3604569 

Ontario, North York, Lawrence Manor
43.7220788 -79.4375067 

['Downtown Toronto' "Queen's Park, Ontario Provincial Government"]
Ontario, Toronto, Queen's Park
43.659659 -79.3903399 

Ontario, Etobicoke, Islington Avenue
43.6794838 -79.5389092 

Ontario, Scarborough, Malvern
43.8091955 -79.2217008 

Ontario, North York, Don Mills
43.775347 -79.3459439 

Ontario, East York, Parkview Hill
43.699971000000005 -79.33251996261595 

['Downtown Toronto' 'Garden District, Ryerson']
Ontario, Toronto, Garden District
43.6564995 -79.3771141 

Ontario, North York, Glencairn
43.7087117 -79.4406853 

Ontario, Etobicoke, West Deane Park
43.6631995 -79.5685684 

Ontario, Scarborough, Rouge Hill
43.7802711 -79.1304992 

Ontario, North York, Don Mills
43.775347 -79.3459439 

Ontario, Eas

In [45]:
dfLocation=pd.DataFrame({'Latitude':lat,'Longitude':long})

In [46]:
dfToronto=dfToronto.reset_index(drop=True)

In [47]:
dfCpy = pd.concat([dfToronto,dfLocation], axis=1)

In [36]:
dfCpy

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.758800,-79.320197
1,M4A,North York,Victoria Village,43.732658,-79.311189
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.660706,-79.360457
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.722079,-79.437507
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.659659,-79.390340
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.679484,-79.538909
6,M1B,Scarborough,"Malvern, Rouge",43.809196,-79.221701
7,M3B,North York,Don Mills,43.775347,-79.345944
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.699971,-79.332520
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.656500,-79.377114


# Part 3: Display Nighborhoods with Folium

### Import the required libraries

In [21]:
pip install folium

Note: you may need to restart the kernel to use updated packages.


In [22]:
import folium

### Display Neighborhoods in a Toronto map

In [48]:
# create map of Toronto using latitude and longitude values
map_Toronto = folium.Map(location=[43.7181557,-79.5181422], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(dfCpy['Latitude'], dfCpy['Longitude'], dfCpy['Borough'], dfCpy['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto